# Notebook for creating GIS features for release risk model

All data is developed at the Census Block Group (CBG) level.

Response: Presence of disposal sites.

Predictors created:
* Num of firestations
* Num of industrial sites
* Land area
* Water Area
* Airports 
* Army Bases 
* Highways (OSM) 
* Population Density 
* Number of Businesses (Look at MassDEP website) 

Features to create:
* Firefighter training facilities 

Maybe stay within CBGs of counties where we have disposal sites - hopefully will do it. Maybe use private well cbgs only. 

****
Notes on feature creation:
- Number of potential sources: Point-within-polygon sum calculation
Disposal site data is skewed towards non-environmental type releases. More along the lines of car crashes, and oils spills. 

****

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import model_utils
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read in CBG polygon as geodataframe
cbg_gdf = gpd.read_file(f'zip://../../data/tl_2010_25_bg10.zip')
cbg_gdf['GEOID10'] = cbg_gdf['GEOID10'].astype(np.int64).astype(str).str.zfill(12)
modeling_dataset = cbg_gdf[['GEOID10', 'NAMELSAD10', 'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10', 'geometry']]

### Response data
* Presence of disposal site in CBG

In [3]:
# Read in disposal site location data
disposal_sites = pd.read_parquet('../../data/disposal_sites/PFAS_Sites_2021-11-07_geocoded.parquet')

# convert coordinate file to gdf
# It does not have a coordinate system - so set it.
# 4326 makes sense if values show longitudes ~ -70 and  latitudes ~ 41
disposal_site_gdf = gpd.GeoDataFrame(
    disposal_sites, 
    geometry=gpd.points_from_xy(disposal_sites.lon, disposal_sites.lat),
    crs={"init":"EPSG:4326"})

# gpd.read_file(f'zip://../../data/c21e_pt.zip.zip!C21E_PT.shp') # Is this the correct file??

response_data = model_utils.sum_points_in_poly(poly_gdf = cbg_gdf, 
                                               point_gdf = disposal_site_gdf,
                                               col_name = 'disposal_sites',
                                               groups = ['GEOID10'])

# Attach to modeling dataset
modeling_dataset = modeling_dataset.merge(response_data, on = 'GEOID10', how = 'left')
modeling_dataset['sum_disposal_sites'] = modeling_dataset['sum_disposal_sites'].replace({np.nan : 0})

# Convert multiple to 1 (since classification problem)
modeling_dataset['response'] = np.where(modeling_dataset['sum_disposal_sites'] >= 1, 1, 0)

In [4]:
modeling_dataset.head(2)

,GEOID10,NAMELSAD10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,sum_disposal_sites,response
0,250235251042,Block Group 2,2648651,119260,+41.9751132,-070.9683373,"POLYGON ((-70.98402 41.97135, -70.98386 41.971...",0.0,0
1,250235251044,Block Group 4,4625818,11563,+41.9677679,-070.9881083,"POLYGON ((-71.00489 41.96832, -71.00486 41.968...",0.0,0


In [5]:
modeling_dataset.describe()

,ALAND10,AWATER10,sum_disposal_sites,response
count,4.985000e+03,4.985000e+03,4985.000000,4985.000000
mean,4.052569e+06,1.431030e+06,0.010231,0.008425
std,1.047237e+07,3.687699e+07,0.120591,0.091411
min,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,2.520430e+05,0.000000e+00,0.000000,0.000000
50%,8.454730e+05,6.313000e+03,0.000000,0.000000
75%,3.188137e+06,1.302360e+05,0.000000,0.000000
max,1.452006e+08,2.152192e+09,3.000000,1.000000


***

## Predictors

Area hierarchies

In [8]:
modeling_dataset['county'] = modeling_dataset['GEOID10'].str[:5]

modeling_dataset['places'] = modeling_dataset['GEOID10'].str[:7]

Firestations

In [9]:
# Firestations
firestation_gdf = gpd.read_file(f'zip://../../data/features/firefacilities_pt.zip')

# Get sum of points in CBG
firestation_data = model_utils.sum_points_in_poly(poly_gdf = cbg_gdf, 
                                                   point_gdf = firestation_gdf,
                                                   col_name = 'firestations',
                                                   groups = ['GEOID10'])

# Attach to modeling dataset
modeling_dataset = modeling_dataset.merge(firestation_data, on = 'GEOID10', how = 'left')
modeling_dataset['sum_firestations'] = modeling_dataset['sum_firestations'].replace({np.nan : 0})

Industry Sources

In [10]:
# Industrial sites
src_pt_df = pd.read_csv(f'../../data/features/BAW PFAS Likely Industry List 04-30-2020 - qryPFAS_High.csv')

In [11]:
# src_pt_df has latitude and longitude incorrect formatting. Fix formatting
src_pt_df['LATITUDE'] = src_pt_df['LATITUDE'].astype(str)
src_pt_df['LONGITUDE'] = src_pt_df['LONGITUDE'].astype(str)
src_pt_df['LATITUDE'] = (src_pt_df['LATITUDE'].str[:2] + '.' + src_pt_df['LATITUDE'].str[2:]).astype(float)
src_pt_df['LONGITUDE'] = ('-' + src_pt_df['LONGITUDE'].str[:2] + '.' + src_pt_df['LONGITUDE'].str[2:]).astype(float)

# convert coordinate file to gdf
# It does not have a coordinate system - so set it.
# 4326 makes sense if values show longitudes ~ -70 and  latitudes ~ 41

src_pt_gdf = gpd.GeoDataFrame(
    src_pt_df, 
    geometry=gpd.points_from_xy(src_pt_df.LONGITUDE, src_pt_df.LATITUDE),
    crs={"init":"EPSG:4326"})

src_pt_data = model_utils.sum_points_in_poly(poly_gdf = cbg_gdf, 
                                           point_gdf = src_pt_gdf,
                                           col_name = 'industry_sites',
                                           groups = ['GEOID10', 'NAICS_DESC'])

# column names for all industries (NAICS_DESC)
industry_cols = src_pt_data.columns

# reset index for merge
src_pt_data = src_pt_data.reset_index()

# Attach to modeling dataset
modeling_dataset = modeling_dataset.merge(src_pt_data, on = 'GEOID10', how = 'left')
modeling_dataset[industry_cols] = modeling_dataset[industry_cols].replace({np.nan : 0})

Population Density

In [12]:
pop_dens_df = pd.read_csv(f'../../data/features/pop_density_cbg_acs_2018_9_2.csv')

pop_dens_df['GEOID10'] = pop_dens_df['GEOID10'].astype(np.int64).astype(str).str.zfill(12)

# Attach to modeling dataset
modeling_dataset = modeling_dataset.merge(pop_dens_df, on = 'GEOID10', how = 'left')
modeling_dataset['pop_density_acs_2018'] = modeling_dataset['pop_density_acs_2018'].replace({np.nan : 0})

Airports

In [13]:
airports_gdf = gpd.read_file(f'zip://../../data/features/Airports.zip')

In [14]:
# Get sum of points in CBG
airports_data = model_utils.sum_points_in_poly(poly_gdf = cbg_gdf, 
                                                   point_gdf = airports_gdf,
                                                   col_name = 'airports',
                                                   groups = ['GEOID10'])

# Attach to modeling dataset
modeling_dataset = modeling_dataset.merge(airports_data, on = 'GEOID10', how = 'left')
modeling_dataset['sum_airports'] = modeling_dataset['sum_airports'].replace({np.nan : 0})

Army Bases

In [15]:
army_bases_gdf = gpd.read_file(f'zip://../../data/features/FY20_MIRTA_Points.zip')

In [16]:
# Get sum of points in CBG
army_data = model_utils.sum_points_in_poly(poly_gdf = cbg_gdf, 
                                                   point_gdf = army_bases_gdf,
                                                   col_name = 'army_bases',
                                                   groups = ['GEOID10'])

# Attach to modeling dataset
modeling_dataset = modeling_dataset.merge(army_data, on = 'GEOID10', how = 'left')
modeling_dataset['sum_army_bases'] = modeling_dataset['sum_army_bases'].replace({np.nan : 0})

Highways

In [17]:
highways_gdf = gpd.read_file(f'zip://../../data/features/CENSUS2010TIGERROADS_ARC.zip')[['TLID', 'geometry']]

In [18]:
line_sum_series = model_utils.sum_lines_in_poly(lines_gdf = highways_gdf, 
                              poly_gdf = cbg_gdf)

In [19]:
modeling_dataset['sum_highways'] = line_sum_series

Parcel Use Codes
* Use codes: https://www.mass.gov/doc/property-type-classification-codes-non-arms-length-codes-and-sales-report-spreadsheet/download
* 97% of parcels are within 1 CBG, so we can just spatially join

In [20]:
tax_parcels_gdf = gpd.read_file(f'zip://../../data/features/TAX_PARCELS.zip')

In [21]:
tax_parcels_gdf = tax_parcels_gdf.to_crs("EPSG:4326")

In [22]:
tax_parcels_gdf['USE_CODE_CLASS_CODE'] = tax_parcels_gdf['USE_CODE'].astype(str).str[:1]

In [23]:
# Create classifications of parcels
mask_multiple_use = (tax_parcels_gdf['USE_CODE_CLASS_CODE'] == '0')
mask_residential = (tax_parcels_gdf['USE_CODE_CLASS_CODE'] == '1')
mask_open_space = (tax_parcels_gdf['USE_CODE_CLASS_CODE'] == '2')
mask_commercial = (tax_parcels_gdf['USE_CODE_CLASS_CODE'] == '3')
mask_industrial = (tax_parcels_gdf['USE_CODE_CLASS_CODE'] == '4')
mask_private_prop = (tax_parcels_gdf['USE_CODE_CLASS_CODE'] == '5')
mask_forest = (tax_parcels_gdf['USE_CODE_CLASS_CODE'] == '6')
mask_agricultural = (tax_parcels_gdf['USE_CODE_CLASS_CODE'] == '7')
mask_recreational = (tax_parcels_gdf['USE_CODE_CLASS_CODE'] == '8')
mask_exempt = (tax_parcels_gdf['USE_CODE_CLASS_CODE'] == '9')

tax_parcels_gdf.loc[mask_multiple_use, 'USE_CODE_CLASS'] = 'MULTIPLE-USE' 
tax_parcels_gdf.loc[mask_residential, 'USE_CODE_CLASS'] = 'RESIDENTIAL' 
tax_parcels_gdf.loc[mask_open_space, 'USE_CODE_CLASS'] = 'OPEN-SPACE' 
tax_parcels_gdf.loc[mask_commercial, 'USE_CODE_CLASS'] = 'COMMERCIAL' 
tax_parcels_gdf.loc[mask_industrial, 'USE_CODE_CLASS'] = 'INDUSTRIAL' 
tax_parcels_gdf.loc[mask_private_prop, 'USE_CODE_CLASS'] = 'PRIVATE-PROPERTY' 
tax_parcels_gdf.loc[mask_forest, 'USE_CODE_CLASS'] = 'FOREST' 
tax_parcels_gdf.loc[mask_agricultural, 'USE_CODE_CLASS'] = 'AGRICULTURAL' 
tax_parcels_gdf.loc[mask_recreational, 'USE_CODE_CLASS'] = 'RECREATIONAL' 
tax_parcels_gdf.loc[mask_exempt, 'USE_CODE_CLASS'] = 'EXEMPT'

In [24]:
# No Private Property data
land_use_cols =['MULTIPLE-USE' 
                ,'RESIDENTIAL' 
                ,'OPEN-SPACE' 
                ,'COMMERCIAL' 
                ,'INDUSTRIAL' 
                ,'FOREST' 
                ,'AGRICULTURAL' 
                , 'RECREATIONAL' 
                ,'EXEMPT']

In [25]:
tp_intersection = tax_parcels_gdf.overlay(cbg_gdf, how = 'intersection')

In [26]:
# Find the area related to each class for each CBG
tp_grp = tp_intersection.groupby(['GEOID10', 'USE_CODE_CLASS'])['SHAPE_AREA'].sum()
tp_grp = pd.DataFrame(tp_grp).reset_index()

In [27]:
# Make into a relative measure
tp_sum = pd.DataFrame(tp_grp.groupby(['GEOID10'])['SHAPE_AREA'].sum()).reset_index()

In [28]:
tp_perc = tp_grp.merge(tp_sum, on = 'GEOID10')

In [29]:
tp_perc['CLASS_AREA_PERC'] = tp_perc['SHAPE_AREA_x'] / tp_perc['SHAPE_AREA_y'] * 100
tp_perc = tp_perc[['GEOID10', 'USE_CODE_CLASS', 'CLASS_AREA_PERC']]

In [30]:
tp_perc = tp_perc.pivot(index='GEOID10', columns='USE_CODE_CLASS', values='CLASS_AREA_PERC').replace({np.nan : 0})
tp_perc = tp_perc.reset_index()

In [31]:
modeling_dataset = modeling_dataset.merge(tp_perc, on = 'GEOID10', how = 'left')
modeling_dataset[land_use_cols] = modeling_dataset[land_use_cols].replace({np.nan : 0})

****
****

### Filter data to relevant areas

In [32]:
private_well_service_areas = gpd.read_file('zip://../../data/private_well_service.zip')

In [33]:
%%time
private_well_service_areas = private_well_service_areas.to_crs("EPSG:4326")

# Intersection to determine CBGs that service private wells

# relevant_cbgs = cbg_gdf.overlay(private_well_service_areas, how = 'intersection')
# relevant_cbgs.to_file('../../data/release_risk_filtering_data_cbgs.geojson', driver='GeoJSON')

Wall time: 1.47 s


In [34]:
relevant_cbgs = gpd.read_file('zip://../../data/release_risk_filtering_data_cbgs.zip')

In [35]:
filter_cbgs = list(relevant_cbgs['GEOID10'])
filtered_modeling_dataset = modeling_dataset[modeling_dataset['GEOID10'].isin(filter_cbgs)]

In [36]:
filtered_modeling_dataset.shape

(2299, 52)

In [37]:
filtered_modeling_dataset.head()

,GEOID10,NAMELSAD10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,sum_disposal_sites,response,county,...,sum_highways,AGRICULTURAL,COMMERCIAL,EXEMPT,FOREST,INDUSTRIAL,MULTIPLE-USE,OPEN-SPACE,RECREATIONAL,RESIDENTIAL
2,250235252031,Block Group 1,2367037,62136,+42.0051872,-070.9629934,"POLYGON ((-70.95998 42.00710, -70.95848 42.006...",0.0,0,25023,...,0.104643,0.000000,2.168028,13.464491,6.605395,0.340688,2.094458,0.0,0.0,75.326940
5,250235101004,Block Group 4,1264335,4841,+42.1159859,-071.0150870,"POLYGON ((-71.02515 42.11744, -71.02228 42.118...",0.0,0,25023,...,0.060472,0.000000,8.777727,18.670316,0.000000,22.785765,0.000000,0.0,0.0,49.766192
6,250235101001,Block Group 1,780921,448,+42.1205088,-071.0063537,"POLYGON ((-71.00199 42.12640, -71.00195 42.126...",0.0,0,25023,...,0.081192,0.000000,0.000000,0.237058,0.000000,0.292122,0.000000,0.0,0.0,99.470820
8,250235021022,Block Group 2,5507262,0,+42.0999824,-070.9011797,"POLYGON ((-70.88879 42.10738, -70.88826 42.099...",0.0,0,25023,...,0.189978,0.587848,0.272041,6.891629,0.000000,1.285071,0.257443,0.0,0.0,90.705968
9,250235062043,Block Group 3,1969145,71242,+42.0959972,-070.6607532,"POLYGON ((-70.65080 42.08668, -70.65165 42.086...",0.0,0,25023,...,0.102837,0.000000,0.209224,8.772981,0.000000,81.733147,0.035358,0.0,0.0,9.249291


***
***

Write out modeling dataset

In [38]:
# Write out as shapefiles
modeling_dataset.to_file(f'../../data/modeling_data/release_risk/full_release_risk_modeling_dataset.shp')
filtered_modeling_dataset.to_file(f'../../data/modeling_data/release_risk/filtered_release_risk_modeling_dataset.shp')

In [41]:
# Write out as normal files
filtered_modeling_dataset.drop(columns = ['geometry'], inplace = True)
modeling_dataset.drop(columns = ['geometry'], inplace = True)

modeling_dataset.to_csv(f'../../data/modeling_data/release_risk/full_release_risk_modeling_dataset.csv', index = False)
filtered_modeling_dataset.to_csv(f'../../data/modeling_data/release_risk/filtered_release_risk_modeling_dataset.csv', index = False)